# Yazan Aakel - 5171606
## Data Visualization Course
* this notebook is for manipulating the data to produce it in a suitable form for drawing the required charts

In [3]:
!pip3 install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 10.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.5/502.5 kB 9.8 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires portpicker==1.5.2, which is not installed.
google-colab 1.0.0 requires google-auth==2.17.3, but you have google-auth 2.22.0 which is incompatible.
google-colab 1.0.0 requires ipykernel==5.5.6, but you have ipykernel 6.26.0 which is incompatible.
google-colab 1.0.0 requires ipython==7.34.0, but you have ipython 8.17.2 which is incompatible.
google-colab 1.0.0 requires notebook==6.5.5, but you have notebook 7.0.6 which is incompatible.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.1.2 which is incompatible.
googl

# Assignment 1 - Task 1 - Finding top 10 tree species with Other in each city - used for vertical and horizontal barcharts

In [28]:
import os
import pandas as pd
import json

def process_data(number_of_top_trees=10):
    # Directory where your CSV files are located
    data_directory = "./Desktop/data-vis"

# Create a directory for the output JSON file
    output_directory = os.path.join(data_directory, "output")
    os.makedirs(output_directory, exist_ok=True)  # Create the directory if it doesn't exist

# Initialize an empty dictionary to store the aggregated data
    aggregated_data = pd.DataFrame(columns=["scientific_name", "common_name", "city", "state", "height_M", "tree_count"])
    aggregated_data['height_M'] = aggregated_data['height_M'].apply(lambda x: x if x > 0 else None)

# List all CSV files in the directory
    csv_files = [file for file in os.listdir(data_directory) if file.endswith(".csv")]

    for csv_file in csv_files:
        print("processing .. " + csv_file)
    
        # Load the data from the CSV file
        data = pd.read_csv(os.path.join(data_directory, csv_file), usecols=['scientific_name', 'city', 'state', 'height_M'])
        #data.dropna(how='any', inplace=True)
    
        df_avg_mean = data.groupby(["scientific_name", "city", "state"])['height_M'].mean().reset_index(name="avg_mean")
        # Group the data by a combination of "city" and "scientific_name" and count the number of trees in each combination
        city_tree_count = data.groupby(["scientific_name", "city", "state"]).size().reset_index(name="tree_count")
        city_tree_count = city_tree_count.merge(df_avg_mean, on=["scientific_name", "city", "state"], how="left")
        
        city_tree_count['avg_mean'] = city_tree_count['avg_mean'].fillna(0)

    # Create a dictionary to store city-specific data
    #city_data = {}

    # Iterate through the sorted data and select the top 10 trees for each city
        for city, city_group in city_tree_count.groupby("city"):
    
            top_trees = city_group.nlargest(number_of_top_trees, "tree_count")
    
            if len(top_trees) < number_of_top_trees:
                continue
            diff = pd.concat([city_group, top_trees]).drop_duplicates(keep=False)
           # print(diff.head(10))
            remaining_count = diff["tree_count"].sum()
    
            remaining_height_avg = diff["avg_mean"].sum()
            # Convert int64 to regular Python int for serialization
            top_trees["tree_count"] = top_trees["tree_count"].astype(int)
            remaining_count = int(remaining_count)
    
    
            #top_trees = top_trees.to_dict(orient="records")
            top_trees = pd.concat([top_trees, pd.DataFrame({'scientific_name': "Other","city":city, "state":city_group['state'],
                                                                  "tree_count": remaining_count, 
                                                                "avg_mean":remaining_height_avg})], ignore_index=True)
    
    
            aggregated_data = pd.concat([aggregated_data, top_trees], ignore_index=True).drop_duplicates(keep='first')

    # Add city-specific data to the aggregated_data dictionary
    #aggregated_data.update(city_data)

# Sort the aggregated_data dictionary by city names alphabetically
#aggregated_data = dict(sorted(aggregated_data.items()))
    aggregated_data.sort_values(by=["tree_count"], inplace=True, ascending=False)

# Save the aggregated JSON data to a single file in the output directory
    output_file = os.path.join(output_directory, "city_tree_counts_top_" + str(number_of_top_trees) + ".csv")
    aggregated_data.to_csv(output_file, index=False)

    cities = aggregated_data["city"].unique()
    print(cities)


    cities_output_file = os.path.join(output_directory, "cities.txt")

    with open(cities_output_file, "w") as citiesOutfile:
        for item in cities:
        # write each item on a new line
            citiesOutfile.write("%s\n" % item)


    states = aggregated_data["state"].unique()
    print(states)

    states_output_file = os.path.join(output_directory, "states.txt")

    with open(states_output_file, "w") as statesOutfile:
        for item in states:
        # write each item on a new line
            statesOutfile.write("%s\n" % item)


In [18]:
# processing for tasks that requires showing data for top 10 trees
process_data(10)

processing .. Milwaukee_Final_2022-06-18.csv
processing .. Anaheim_Final_2022-06-18.csv
processing .. Irvine_Final_2022-06-18.csv
processing .. RanchoCucamonga_Final_2022-06-18.csv
processing .. Arlington_Final_2022-06-18.csv
processing .. Rochester_Final_2022-06-18.csv
processing .. Miami_Final_2022-06-18.csv
processing .. Austin_Final_2022-06-18.csv
processing .. Stockton_Final_2022-06-18.csv
processing .. Seattle_Final_2022-06-18.csv
processing .. LosAngeles_Final_2022-06-18.csv
processing .. AuroraCO_Final_2022-06-18.csv
processing .. NewYork_Final_2022-06-18.csv
processing .. Oakland_Final_2022-06-18.csv
processing .. SanFrancisco_Final_2022-06-18.csv
processing .. Boston_Final_2022-06-18.csv
processing .. Ontario_Final_2022-06-18.csv
processing .. GrandRapids_Final_2022-06-18.csv
processing .. Baltimore_Final_2022-06-18.csv
processing .. Albuquerque_Final_2022-06-18.csv
processing .. Greensboro_Final_2022-06-18.csv
processing .. OklahomaCity_Final_2022-06-18.csv
processing .. Hun

In [29]:
process_data(5)

processing .. Milwaukee_Final_2022-06-18.csv
processing .. Anaheim_Final_2022-06-18.csv
processing .. Irvine_Final_2022-06-18.csv
processing .. RanchoCucamonga_Final_2022-06-18.csv
processing .. Arlington_Final_2022-06-18.csv
processing .. Rochester_Final_2022-06-18.csv
processing .. Miami_Final_2022-06-18.csv
processing .. Austin_Final_2022-06-18.csv
processing .. Stockton_Final_2022-06-18.csv
processing .. Seattle_Final_2022-06-18.csv
processing .. LosAngeles_Final_2022-06-18.csv
processing .. AuroraCO_Final_2022-06-18.csv
processing .. NewYork_Final_2022-06-18.csv
processing .. Oakland_Final_2022-06-18.csv
processing .. SanFrancisco_Final_2022-06-18.csv
processing .. Boston_Final_2022-06-18.csv
processing .. Ontario_Final_2022-06-18.csv
processing .. GrandRapids_Final_2022-06-18.csv
processing .. Baltimore_Final_2022-06-18.csv
processing .. Albuquerque_Final_2022-06-18.csv
processing .. Greensboro_Final_2022-06-18.csv
processing .. OklahomaCity_Final_2022-06-18.csv
processing .. Hun

# This is for heatmap, eliminating the cities that does not have a certain type of trees

In [51]:
import os
import pandas as pd

# Directory where your CSV files are located
data_directory = "./Desktop/data-vis"

# Create a directory for the output CSV file
output_directory = os.path.join(data_directory, "output")
os.makedirs(output_directory, exist_ok=True)  # Create the directory if it doesn't exist

# Initialize an empty dictionary to store the aggregated data
aggregated_data = {}

# List all CSV files in the directory
csv_files = [file for file in os.listdir(data_directory) if file.endswith(".csv")]

# Define the number of top tree species to consider
top_tree_count = 5

for csv_file in csv_files:
    print("Processing " + csv_file)

    # Load the data from the CSV file
    data = pd.read_csv(os.path.join(data_directory, csv_file), usecols=['state', 'city', 'common_name', 'scientific_name'])

    # Group the data by state, city, and common_name, and count the number of trees in each combination
    city_tree_count = data.groupby(['state', 'city', 'scientific_name']).size().reset_index(name="tree_count")

    # Sort the data in descending order based on tree_count
    sorted_city_tree_count = city_tree_count.sort_values(by=['state', 'city', 'tree_count'], ascending=[True, True, False])

    # Create a dictionary to store the top tree occurrences for each city
    city_data = {}

    for (state, city), group in sorted_city_tree_count.groupby(['state', 'city']):
        top_trees = group.head(top_tree_count)
        diff = pd.concat([group, top_trees]).drop_duplicates(keep=False)
        remaining_count = diff["tree_count"].sum()

        
        top_trees = pd.concat([top_trees, pd.DataFrame({"state": group['state'], "city": city, 'scientific_name': "Other",
                                                       "tree_count": remaining_count})]).drop_duplicates(keep='last')

        tree_counts = top_trees['tree_count'].tolist()
        city_data[(state, city)] = tree_counts

    # Add the city-specific data to the aggregated_data dictionary
    aggregated_data.update(city_data)

# Create a list to store the CSV data
csv_data = []

# Define column names for the CSV
column_names = ['city', 'state', 'scientific_name', 'tree_count']

# Iterate through the aggregated data and format it for CSV
for (state, city), tree_counts in aggregated_data.items():
    for scientific_name, tree_count in zip(top_trees['scientific_name'], tree_counts):
        csv_data.append([city, state, scientific_name, tree_count])

# Create a DataFrame from the CSV data
csv_df = pd.DataFrame(csv_data, columns=column_names)

# Filter cities that have at least one of each of the top 5 species
filtered_df = csv_df.groupby(['city', 'state']).filter(lambda x: x['scientific_name'].nunique() >= top_tree_count)

# Save the aggregated data to a CSV file
output_file = os.path.join(output_directory, "city_tree_counts_top_5.csv")
filtered_df.to_csv(output_file, index=False)


Processing Milwaukee_Final_2022-06-18.csv
Processing Anaheim_Final_2022-06-18.csv
Processing Irvine_Final_2022-06-18.csv
Processing RanchoCucamonga_Final_2022-06-18.csv
Processing Arlington_Final_2022-06-18.csv
Processing Rochester_Final_2022-06-18.csv
Processing Miami_Final_2022-06-18.csv
Processing Austin_Final_2022-06-18.csv
Processing Stockton_Final_2022-06-18.csv
Processing Seattle_Final_2022-06-18.csv
Processing LosAngeles_Final_2022-06-18.csv
Processing AuroraCO_Final_2022-06-18.csv
Processing NewYork_Final_2022-06-18.csv
Processing Oakland_Final_2022-06-18.csv
Processing SanFrancisco_Final_2022-06-18.csv
Processing Boston_Final_2022-06-18.csv
Processing Ontario_Final_2022-06-18.csv
Processing GrandRapids_Final_2022-06-18.csv
Processing Baltimore_Final_2022-06-18.csv
Processing Albuquerque_Final_2022-06-18.csv
Processing Greensboro_Final_2022-06-18.csv
Processing OklahomaCity_Final_2022-06-18.csv
Processing HuntingtonBeach_Final_2022-06-18.csv
Processing Fresno_Final_2022-06-18.

# For Stacked Barcharts

In [50]:
import os
import pandas as pd

# Directory where your CSV files are located
data_directory = "./Desktop/data-vis"

# Create a directory for the output CSV file
output_directory = os.path.join(data_directory, "output")
os.makedirs(output_directory, exist_ok=True)  # Create the directory if it doesn't exist

# Initialize an empty dictionary to store the aggregated data
aggregated_data = {}

# List all CSV files in the directory
csv_files = [file for file in os.listdir(data_directory) if file.endswith(".csv")]

# Define the number of top tree species to consider
top_tree_count = 5

for csv_file in csv_files:
    print("Processing " + csv_file)

    # Load the data from the CSV file
    data = pd.read_csv(os.path.join(data_directory, csv_file), usecols=['state', 'city', 'common_name', 'scientific_name'])

    # Group the data by state, city, and common_name, and count the number of trees in each combination
    city_tree_count = data.groupby(['state', 'city', 'scientific_name']).size().reset_index(name="tree_count")

    # Sort the data in descending order based on tree_count
    sorted_city_tree_count = city_tree_count.sort_values(by=['state', 'city', 'tree_count'], ascending=[True, True, False])

    # Create a dictionary to store the top tree occurrences for each city
    city_data = {}

    for (state, city), group in sorted_city_tree_count.groupby(['state', 'city']):
        top_trees = group.head(top_tree_count)
        diff = pd.concat([group, top_trees]).drop_duplicates(keep=False)
        remaining_count = diff["tree_count"].sum()
       
        top_trees = pd.concat([top_trees, pd.DataFrame({"state":group['state'], "city":city, 'scientific_name': "Other",
                                                              "tree_count": remaining_count})]).drop_duplicates(keep='last')

        tree_counts = top_trees['tree_count'].tolist()
        city_data[(state, city)] = tree_counts

    # Add the city-specific data to the aggregated_data dictionary
    aggregated_data.update(city_data)
# Create a list to store the CSV data
csv_data = []

# Define column names for the CSV
column_names = ['state', 'city'] + [f'{row["scientific_name"]}' for index, row in top_trees.iterrows()]

# Iterate through the aggregated data and format it for CSV
for (state, city), tree_counts in aggregated_data.items():
    csv_row = [state, city] + tree_counts
    csv_data.append(csv_row)

# Create a DataFrame from the CSV data
csv_df = pd.DataFrame(csv_data, columns= column_names)

# Save the aggregated data to a CSV file
output_file = os.path.join(output_directory, "tree_occurrences_with_city.csv")
csv_df.to_csv(output_file, index=False)

print("CSV file saved:", output_file)


Processing Milwaukee_Final_2022-06-18.csv
Processing Anaheim_Final_2022-06-18.csv
Processing Irvine_Final_2022-06-18.csv
Processing RanchoCucamonga_Final_2022-06-18.csv
Processing Arlington_Final_2022-06-18.csv
Processing Rochester_Final_2022-06-18.csv
Processing Miami_Final_2022-06-18.csv
Processing Austin_Final_2022-06-18.csv
Processing Stockton_Final_2022-06-18.csv
Processing Seattle_Final_2022-06-18.csv
Processing LosAngeles_Final_2022-06-18.csv
Processing AuroraCO_Final_2022-06-18.csv
Processing NewYork_Final_2022-06-18.csv
Processing Oakland_Final_2022-06-18.csv
Processing SanFrancisco_Final_2022-06-18.csv
Processing Boston_Final_2022-06-18.csv
Processing Ontario_Final_2022-06-18.csv
Processing GrandRapids_Final_2022-06-18.csv
Processing Baltimore_Final_2022-06-18.csv
Processing Albuquerque_Final_2022-06-18.csv
Processing Greensboro_Final_2022-06-18.csv
Processing OklahomaCity_Final_2022-06-18.csv
Processing HuntingtonBeach_Final_2022-06-18.csv
Processing Fresno_Final_2022-06-18.

In [14]:
import os
import pandas as pd

# Directory where your CSV files are located
data_directory = "./Desktop/data-vis"

# Create a directory for the output CSV file
output_directory = os.path.join(data_directory, "output")
os.makedirs(output_directory, exist_ok=True)  # Create the directory if it doesn't exist

# Initialize an empty DataFrame to store the Sankey data
sankey_df = pd.DataFrame(columns=['state', 'city', 'scientific_name', 'count'])

# List all CSV files in the directory
csv_files = [file for file in os.listdir(data_directory) if file.endswith(".csv")]

# Define the number of top tree species to consider
top_tree_count = 5

for csv_file in csv_files:
    print("Processing " + csv_file)

    # Load the data from the CSV file
    data = pd.read_csv(os.path.join(data_directory, csv_file), usecols=['state', 'city', 'scientific_name', 'scientific_name'])

    # Group the data by state, city, and common_name, and count the number of trees in each combination
    city_tree_count = data.groupby(['state', 'city', 'scientific_name']).size().reset_index(name="count")

    # Sort the data in descending order based on count
    sorted_city_tree_count = city_tree_count.sort_values(by=['state', 'city', 'count'], ascending=[True, True, False])

    # Create a dictionary to store the top tree occurrences for each city
    city_data = {}

    for (state, city), group in sorted_city_tree_count.groupby(['state', 'city']):
        top_trees = group.head(top_tree_count)
        
        # Append the data to the Sankey DataFrame
        sankey_df = pd.concat([sankey_df, top_trees[['state', 'city', 'scientific_name', 'count']]])

# Save the Sankey DataFrame to a CSV file
output_file = os.path.join(output_directory, "sankey_data.csv")
sankey_df.to_csv(output_file, index=False)

print("CSV file saved:", output_file)


Processing Milwaukee_Final_2022-06-18.csv
Processing Anaheim_Final_2022-06-18.csv
Processing Irvine_Final_2022-06-18.csv
Processing RanchoCucamonga_Final_2022-06-18.csv
Processing Arlington_Final_2022-06-18.csv
Processing Rochester_Final_2022-06-18.csv
Processing Miami_Final_2022-06-18.csv
Processing Austin_Final_2022-06-18.csv
Processing Stockton_Final_2022-06-18.csv
Processing Seattle_Final_2022-06-18.csv
Processing LosAngeles_Final_2022-06-18.csv
Processing AuroraCO_Final_2022-06-18.csv
Processing NewYork_Final_2022-06-18.csv
Processing Oakland_Final_2022-06-18.csv
Processing SanFrancisco_Final_2022-06-18.csv
Processing Boston_Final_2022-06-18.csv
Processing Ontario_Final_2022-06-18.csv
Processing GrandRapids_Final_2022-06-18.csv
Processing Baltimore_Final_2022-06-18.csv
Processing Albuquerque_Final_2022-06-18.csv
Processing Greensboro_Final_2022-06-18.csv
Processing OklahomaCity_Final_2022-06-18.csv
Processing HuntingtonBeach_Final_2022-06-18.csv
Processing Fresno_Final_2022-06-18.

Python 3.10.8
